In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import yaml

from dist.models import fish, irri, hydro

In [ ]:
models = {
    "fish": fish.model,
    "irri": irri.model,
    "hydro": hydro.model,
}

In [ ]:
def load_data(name):
    gdf = gpd.read_file(f"dist/data/{name}.geojson")
    config = yaml.safe_load(open(f"dist/config/{name}.yml"))
    pars = {p["col"]: p["val"] for p in config["pars"]}
    return gdf, pars

In [ ]:
def process(gdf, pars, model):
    return pd.concat(
        [gdf, pd.DataFrame([model(row.to_dict(), pars) for _, row in gdf.iterrows()])],
        axis="columns",
    )

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 10))
for i, name in enumerate(models.keys()):
    model = models[name]
    gdf, pars = load_data(name)
    out = process(gdf, pars, model)
    out.plot(ax=ax[i], column="tech")
plt.show()